In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
import os
from bs4 import BeautifulSoup as bs
from time import sleep

In [ ]:
#Generates a text file containing all of the pdf urls that will be downloaded
def getPDFURLs(endingPageNumber,outputPath):
    # add headers to prevent 502 request error
    headers = {
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'}

    f = open(outputPath,'w')
    f.close()

    baseUrl = "http://www.brocktonpolice.com/category/police-log/page/"

    pageNumber = endingPageNumber
    # check if file exists and create it if it does not
    if not os.path.isfile(outputPath):
        f.open(outputPath,'w')
        f.close()

    
    f = open(outputPath, 'a')

    while(pageNumber > 0):
        fullPath = baseUrl+str(pageNumber)+"/"
        
        data = requests.get(fullPath, headers=headers, stream=True)    
        soup = bs(data.content, 'html.parser')
        basePageLinks = []
        # Get all links on the first page
        for e in soup.find_all('h2'):
            for link in e.find_all('a', href=True):
                basePageLinks.append(link['href'])
        # Follow the links from the first page to the second page and record the PDF urls
        for link in basePageLinks:
            data = requests.get(link, headers=headers, stream=True)
            soup = bs(data.content, 'html.parser')
            for e in soup.find_all('article'):
                for l in e.find_all('a', href=True):
                    if '.pdf' in l['href']:
                        f.write(l['href']+'\n')

        pageNumber -= 1

    f.close()

In [ ]:
# downloads all of the pdfs from the list of links previously recorded
def downloadPDFs(pdfLinks,outputPath):
    # add headers to prevent 502 request error
    headers = {
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'}

    # ChromeDriver requires backslashes otherwise the download is not automatic and a popup is generated
    fp = outputPath.replace("/","\\")

    if not os.path.exists(fp):
        os.makedirs(fp)
    
    print("Downloading to",fp)

    # Set auto download options
    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
    "download.default_directory": fp,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True})

    driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
    
    for link in pdfLinks:
        driver.get(link)
        sleep(7)
    
    driver.quit()

# Generate a Text File That Holds the URLs where the PDFs are Located

In [ ]:
outputPath = "./brocktonPDFUrls.txt"

# get the most recent n pages of data
getPDFURLs(27,outputPath)

# Download the PDFs Based on the Links in the Text File

In [ ]:
outputPath = "./PDFs"
pdfs = []

f = open("./brocktonPDFUrls.txt",'r')
for line in f:
    pdfs.append(line.strip('\n'))

f.close()

downloadPDFs(pdfs,outputPath)